In [52]:
import blaze as bz
import numpy as np
import pandas as pd
import json
from datashape import dshape

from zipline.data import bundles
from zipline.utils.calendars import get_calendar

from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.data import DataSet
from zipline.pipeline.engine import SimplePipelineEngine
from zipline.pipeline.filters import StaticAssets
from zipline.pipeline.loaders import USEquityPricingLoader
from zipline.pipeline.loaders.blaze import BlazeLoader, from_blaze

In [53]:
trading_calendar = get_calendar('NYSE')
bundle_data = bundles.load('quandl')

In [54]:
with open('../alphatools/data/factory/data_sources.json') as f:
    data_sources = json.load(f)

In [55]:
pwd

u'/Users/jonathan/devwork/alphatools/notebooks'

In [56]:
loc = data_sources.keys()[0]
shape = dshape(data_sources[loc]['schema'])

In [57]:
expr = bz.data(
    '../alphatools/data/factory/' + loc,
    dshape=shape
)

In [58]:
expr

,asof_date,sid,value
0,2010-01-01,0,0.543405
1,2010-01-02,1,0.278369
2,2010-01-03,2,0.424518
3,2010-01-04,3,0.844776
4,2010-01-05,4,0.004719
5,2010-01-06,5,0.121569
6,2010-01-07,6,0.670749
7,2010-01-08,7,0.825853
8,2010-01-09,8,0.136707
9,2010-01-10,9,0.575093


In [59]:
expr.schema

dshape("{asof_date: datetime, sid: int64, value: float64}")

In [60]:
# create and empty BlazeLoader
my_blaze_loader = BlazeLoader()

# create the DataSet
ds = from_blaze(
    expr,
    no_deltas_rule='ignore',
    no_checkpoints_rule='ignore',
    loader=my_blaze_loader,
    missing_values={'index':-1}
)

In [61]:
def choose_loader(column):
    return my_blaze_loader

In [62]:
engine = SimplePipelineEngine(
    get_loader=choose_loader,
    calendar=trading_calendar.all_sessions,
    asset_finder=bundle_data.asset_finder,
)

In [63]:
assets = bundle_data.asset_finder.lookup_symbols(['A', 'AAL'], as_of_date=None)

In [64]:
p = Pipeline(
    columns={
        'col_A': ds.value.latest,
    },
    screen=StaticAssets(assets)
)

In [66]:
df = engine.run_pipeline(
    p,
    pd.Timestamp('2016-01-05', tz='utc'),
    pd.Timestamp('2018-01-04', tz='utc')
)

In [67]:
df.head()

col_A
2016-01-05 00:00:00+00:00 Equity(0 [A])    0.543405
                          Equity(2 [AAL])  0.424518
2016-01-06 00:00:00+00:00 Equity(0 [A])    0.543405
                          Equity(2 [AAL])  0.424518
2016-01-07 00:00:00+00:00 Equity(0 [A])    0.543405